In [ ]:
#carregando as bibliotecas
import pandas as pd
import torch
import transformers
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [ ]:
# carregando o dataset
df = pd.read_csv("yourfile)

# dividindo o texto para samples
train_text, test_text, train_labels, test_labels = train_test_split(df["Text"], df["Dummy Populismo"], test_size=0.2)

In [ ]:
# carregar o tokenizador do bert
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# transformar em tokens
train_tokens = tokenizer.batch_encode_plus(train_text.tolist(), max_length=128, pad_to_max_length=True, truncation=True)
test_tokens = tokenizer.batch_encode_plus(test_text.tolist(), max_length=128, pad_to_max_length=True, truncation=True)

# alterando para torch
train_seq = torch.tensor(train_tokens["input_ids"])
train_mask = torch.tensor(train_tokens["attention_mask"])
train_labels = torch.tensor(train_labels.tolist())

test_seq = torch.tensor(test_tokens["input_ids"])
test_mask = torch.tensor(test_tokens["attention_mask"])
test_labels = torch.tensor(test_labels.tolist())

# carregando o modelo pré-treinado bert
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# definindo o AdamW e a lr
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceCla

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# criando o dataloader
batch_size = 5
train_data = TensorDataset(train_seq, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
test_data = TensorDataset(test_seq, test_mask, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
#treinando
model.train()
for epoch in range(epochs):
    for batch in train_dataloader:
        batch_seq = batch[0].to(device)
        batch_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()
        logits = model(batch_seq, attention_mask=batch_mask)[0]
        loss = criterion(logits, batch_labels)
        loss.backward()
        optimizer.step()

# avaliando
model.eval()
with torch.no_grad():
    test_seq = test_seq.to(device)
    test_mask = test_mask


In [ ]:
# avaliando parâmetros de precisão, acurácia, f1

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
with torch.no_grad():
    test_seq = test_seq.to(device)
    test_mask = test_mask.to(device)
    logits = model(test_seq, attention_mask=test_mask)
    predictions = torch.argmax(logits[0], axis=1)
    accuracy = accuracy_score(test_labels, predictions.numpy())
    precision = precision_score(test_labels, predictions.numpy(), average='weighted')
    recall = recall_score(test_labels, predictions.numpy(), average='weighted')
    f1 = f1_score(test_labels, predictions.numpy(), average='weighted')

print("Accuracy: {:.3f}, Precision: {:.3f}, Recall: {:.3f}, F1: {:.3f}".format(accuracy, precision, recall, f1))


Accuracy: 0.697, Precision: 0.720, Recall: 0.697, F1: 0.704


In [ ]:
#comparando a previsão com o resultado final
for i in range(len(test_text)):
    print("Tweet: {}".format(test_text.iloc[i]))
    print("Predicted label: {}".format(predictions[i]))
    print("Actual label: {}".format(test_labels[i]))

Tweet:     s_romanisio Porraa isso aqui é Elite caralho!!
Predicted label: 0
Actual label: 0
Tweet:     senadorhumberto A isenção deveria ser de pelo menos 7000 reais para dar um fôlego para os micro empresários, bem todos os trabalhadores são de carteira assinada, tem muita gente que trabalha autônomo, vcs poderiam começar a favorecer estás pessoas
Predicted label: 1
Actual label: 0
Tweet:     LUIZPATRIOTA39 O mercado está criticando, Luiz!
Eles não aceitam esse salário mínimo caro e o aumento das bolsas!
Segundo a faria lima, basta ter sol, água e força de vontade.
Predicted label: 1
Actual label: 0
Tweet:     NiltoTatto     guilhermezmazi1     TheInterceptBr Ah mas são tão bonzinhos ,uma minoria de esquerda diz que são trabalhadores coitados. Aff prende esses vagab....
Predicted label: 0
Actual label: 1
Tweet: O POVO CANSOU  DOS URUBUS, CARNICEIROS, CORRUPTOS, LADRÕES, TRAIDORES DA PÁTRIA!
O STE E STF PRECISAM DE MINISTROS COMPETENTES VOTADO PELO POVO!
CHEGA DE INDICAÇÃO. QUEREMOS E